In [ ]:
# GPU: 32*40 in 8.00s = 160/s
# CPU: 32*8 in 115.0s = 2/s

In [ ]:
using Pkg; installed = Pkg.installed()
for p in ("Knet", "KnetLayers", "Images", "BSON")
    haskey(installed,p) || Pkg.add(p)
end
include("./resnetlib.jl")
gpu()

In [ ]:
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.installed()["Knet"])
println("Cores: ", chop(read(pipeline(`cat /proc/cpuinfo`,`grep processor`,`wc -l`),String)))
print("GPUs:\n", read(`nvidia-smi --query-gpu=gpu_name --format=csv,noheader`,String))

In [ ]:
const BATCH_SIZE = 32
const RESNET_FEATURES = 2048
const BATCHES_GPU = 40
const BATCHES_CPU = 8

In [ ]:
# Create batches of fake data
function fakedata(batches; atype=KnetArray)
    x = rand(Float32, 224, 224, 3, BATCH_SIZE * batches)
    minibatch(x, BATCH_SIZE, xtype=atype)
end

In [ ]:
# Return features from classifier
function predictfn(weights, moments, data)
    out = []
    for x in data
        pred = resnet50(weights, moments, x; stage=5)
        push!(out, mat(pred))
    end
    return Array(hcat(out...))
end

## 1. GPU

In [ ]:
# Initialize resnet weights and fake data
gpuweights = gpumoments = nothing; Knet.gc() # clear memory from previous run
KnetLayers.settype!(KnetArray{Float32})
gpuweights, gpumoments = trained(ResNet50);

In [ ]:
@info("Cold start")
gpudata1 = fakedata(BATCHES_GPU, atype=KnetArray)
@time predictfn(gpuweights, gpumoments, gpudata1);

In [ ]:
@info("Benchmarking")
gpudata = fakedata(BATCHES_GPU, atype=KnetArray)
@time predictfn(gpuweights, gpumoments, gpudata);

## 2. CPU

In [ ]:
# Initialize resnet weights
KnetLayers.settype!(Array{Float32})
cpuweights, cpumoments = resnet50init(;stage=5, trained=true, atype=Array);

In [ ]:
@info("Cold start")
cpudata1 = fakedata(1, atype=Array);
@time predictfn(cpuweights, cpumoments, cpudata1);

In [ ]:
@info("Benchmarking")
cpudata = fakedata(BATCHES_CPU, atype=Array);
@time predictfn(cpuweights, cpumoments, cpudata);